In [39]:
import pandas as pd
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
import re
from collections import Counter

In [76]:
def remove_repeated_chars(text):
    return re.sub(r'(.)\1+', r'\1', text)  # 연속된 문자를 하나로 축소

def clean_review_column(df, column_name):
    # '\n'을 제거하고 한글 자모 및 구두점 제거
    df[column_name] = df[column_name].str.replace('\n', ' ', regex=False)  # 개행 문자 제거
    df[column_name] = df[column_name].str.replace(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', regex=True)  # 한글 자모 제거
    df[column_name] = df[column_name].str.replace(r'[^\w\s]', ' ', regex=True)  # 특정 구두점 제거
    df[column_name] = df[column_name].str.replace(r'\d+', ' ', regex=True)  # 숫자 제거
    df[column_name] = df[column_name].apply(remove_repeated_chars) # 연속된 문자 제거
    df[column_name] = df[column_name].str.replace(r'\s+', ' ', regex=True)  # 여러 개의 공백을 한 칸으로 변환
    df = df.drop_duplicates(subset=[column_name], keep='first')  # 중복된 리뷰 제거, 첫 번째만 남김
    
    return df

def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()  # 각 줄마다 불용어를 리스트에 저장
    return stopwords

# # 각 문서에서 핵심 키워드 추출 ( 유사도 포함)
# def extract_keywords(text):
#     # 각 텍스트에서 상위 5개의 키워드 추출
#     keywords = kw_model.extract_keywords(text, vectorizer=vectorizer, top_n=5)
#     return keywords

# 키워드 추출 함수 정의 (유사도 제거 및 예외 처리)
def extract_keywords(text, kw_model, vectorizer):
    # 불용어 처리 후 텍스트가 비어있는지 확인
    if not text.strip():
        return []  # 빈 텍스트는 건너뛰기
    
    try:
        # 키워드 추출
        keywords = kw_model.extract_keywords(text, vectorizer=vectorizer, top_n=10)
        return [keyword for keyword, score in keywords]  # 유사도 점수는 제외하고 키워드만 반환
    except ValueError as e:
        # 빈 문서나 유효한 단어가 없을 경우 예외 처리
        print(f"Skipping document due to error: {e}")
        return []

# def lemmatize_keyword(keyword):
#     morphs = okt.pos(keyword, stem=True)  # 형태소 분석 및 어간 추출
#     lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']] # 명사, 동사, 형용사 추출
#     return ' '.join(lemmatized) if lemmatized else None  # 원형 변환된 단어들을 합침, 없으면 None 반환
    

# 키워드를 원형으로 변환하는 함수
def lemmatize_keyword(keyword, okt, stop_words):
    morphs = okt.pos(keyword, stem=True)  # 형태소 분석 및 어간 추출
    lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective'] and word not in stop_words]
    return ' '.join(lemmatized) if lemmatized else None  # 원형 변환된 단어들을 합침, 없으면 None 반환



In [50]:
# 메인 함수 (파일명 리스트를 CSV 파일에서 가져와 처리)
def process_keywords_from_file_list(file_list_csv, output_csv, stopword_file):
    # 불용어 파일 불러오기
    stop_words = load_stopwords(stopword_file)

    # 파일명 목록 CSV 불러오기
    file_df = pd.read_csv(file_list_csv)

    # KeyBERT 모델 초기화
    kw_model = KeyBERT()
    vectorizer = CountVectorizer(ngram_range=(1, 1))

    # Okt 형태소 분석기 초기화
    okt = Okt()

    # 기존 output CSV 파일이 있는지 확인하고 로드
    try:
        results_df = pd.read_csv(output_csv)
        print(f"Existing results loaded from {output_csv}")
    except FileNotFoundError:
        # 없을 경우 새로운 데이터프레임 생성
        results_df = pd.DataFrame(columns=['title', 'keyword'])
        print(f"No existing results. Creating new results file: {output_csv}")

    # 파일 처리 시작
    for idx, row in file_df.iterrows():
        file_name = row['File Name']  # 파일명 불러오기
        csv_title = file_name.rsplit('.', 1)[0]  # 파일명에서 확장자 제거하여 제목으로 사용
        file_path = os.path.join('reviews', file_name)  # reviews 폴더 내의 파일 경로 설정

        # 파일 경로 출력
        print(f"Trying to load file from: {file_path}")

        # 중복된 파일 처리 방지
        if csv_title in results_df['title'].values:
            print(f"Skipping {file_name}, already processed.")
            continue

        print(f"Processing file: {file_path}")

        # 데이터 로드 및 정제
        try:
            df = pd.read_csv(file_path)
            print(f"Successfully loaded {file_name}")
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")
            continue

        # 리뷰 데이터 정제
        df = clean_review_column(df, 'Review')
        documents = df['Review']

        # 전체 문서에 대해 핵심 키워드 추출
        df['Keywords'] = documents.apply(lambda text: extract_keywords(text, kw_model, vectorizer))
        print(f"Keywords extracted from {file_name}")

        # 키워드를 모두 모아서 리스트로 풀기
        all_keywords = []
        for keywords in df['Keywords']:
            for keyword in keywords:
                lemmatized_keyword = lemmatize_keyword(keyword, okt, stop_words)
                if lemmatized_keyword:  # 불용어 필터링
                    all_keywords.append(lemmatized_keyword)

        # 각 키워드의 빈도를 계산하고 상위 10개 키워드 추출
        keyword_counts = Counter(all_keywords)
        top_10_keywords = keyword_counts.most_common(10)

        # 상위 10개의 키워드를 쉼표로 구분하여 문자열로 변환
        top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]
        keywords_str = ', '.join(top_10_keywords_list)

        # 새로운 데이터프레임 행 생성
        new_row = pd.DataFrame({'title': [csv_title], 'keyword': [keywords_str]})

        # 데이터프레임에 새로운 행 추가 (concat 사용)
        results_df = pd.concat([results_df, new_row], ignore_index=True)

    # 결과를 CSV 파일로 저장
    results_df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

In [51]:
# 사용 예시
file_list_csv = 'list_tshirts.csv'  # 파일명 목록이 담긴 CSV 파일
output_csv = 'output_tshirts.csv'  # 결과를 저장할 CSV 파일
stopword_file = '한국어불용어.txt'  # 불용어 목록 파일

process_keywords_from_file_list(file_list_csv, output_csv, stopword_file)

Existing results loaded from output_tshirts.csv
Trying to load file from: reviews\DOODLE HEART HALF T WHITE GREYISH BLUE.csv
Skipping DOODLE HEART HALF T WHITE GREYISH BLUE.csv, already processed.
Trying to load file from: reviews\[SET] 워셔블 케이블 반팔 니트 세트.csv
Skipping [SET] 워셔블 케이블 반팔 니트 세트.csv, already processed.
Trying to load file from: reviews\[2PACK] 쿨 코튼 티셔츠 블랙+화이트.csv
Skipping [2PACK] 쿨 코튼 티셔츠 블랙+화이트.csv, already processed.
Trying to load file from: reviews\[3천원 결제혜택]링클 체크 박시 오버핏 롤업 하프 셔츠 다크 네이비.csv
Skipping [3천원 결제혜택]링클 체크 박시 오버핏 롤업 하프 셔츠 다크 네이비.csv, already processed.
Trying to load file from: reviews\460G 컷 헤비 피그먼트 티셔츠-차콜-.csv
Skipping 460G 컷 헤비 피그먼트 티셔츠-차콜-.csv, already processed.
Trying to load file from: reviews\[16수 코마사] 2PACK SMALL ARCH T-SHIRT WHITE _ BLACK.csv
Skipping [16수 코마사] 2PACK SMALL ARCH T-SHIRT WHITE _ BLACK.csv, already processed.
Trying to load file from: reviews\빈티지 워싱 네이비 체크셔츠.csv
Skipping 빈티지 워싱 네이비 체크셔츠.csv, already processed.
Trying to load file from: rev

### 바지 한정

In [80]:
# 메인 함수 (파일명 리스트를 CSV 파일에서 가져와 처리)
def process_keywords_from_pants(file_list_csv, output_csv, stopword_file):
    # 불용어 파일 불러오기
    stop_words = load_stopwords(stopword_file)

    # 파일명 목록 CSV 불러오기
    file_df = pd.read_csv(file_list_csv)

    # KeyBERT 모델 초기화
    kw_model = KeyBERT()
    vectorizer = CountVectorizer(ngram_range=(1, 1))

    # Okt 형태소 분석기 초기화
    okt = Okt()

    # 기존 output CSV 파일이 있는지 확인하고 로드
    try:
        results_df = pd.read_csv(output_csv)
        print(f"Existing results loaded from {output_csv}")
    except FileNotFoundError:
        # 없을 경우 새로운 데이터프레임 생성
        results_df = pd.DataFrame(columns=['title', 'keyword'])
        print(f"No existing results. Creating new results file: {output_csv}")

    # 파일 처리 시작
    for idx, row in file_df.iterrows():
        file_name = row['File Name']  # 파일명 불러오기
        csv_title = file_name.rsplit('.', 1)[0]  # 파일명에서 확장자 제거하여 제목으로 사용
        file_path = os.path.join('reviews', file_name)  # reviews 폴더 내의 파일 경로 설정

        # 파일 경로 출력
        print(f"Trying to load file from: {file_path}")

        # 중복된 파일 처리 방지
        if csv_title in results_df['title'].values:
            print(f"Skipping {file_name}, already processed.")
            continue

        print(f"Processing file: {file_path}")

        # 데이터 로드 및 정제
        try:
            df = pd.read_csv(file_path)
            print(f"Successfully loaded {file_name}")
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")
            continue

        # 리뷰 데이터 정제
        df = clean_review_column(df, 'Review')
        documents = df['Review']

        # 전체 문서에 대해 핵심 키워드 추출
        df['Keywords'] = documents.apply(lambda text: extract_keywords(text, kw_model, vectorizer))
        print(f"Keywords extracted from {file_name}")

        # 키워드를 모두 모아서 리스트로 풀기
        all_keywords = []
        for keywords in df['Keywords']:
            for keyword in keywords:
                lemmatized_keyword = lemmatize_keyword(keyword, okt, stop_words)
                if lemmatized_keyword:  # 불용어 필터링
                    all_keywords.append(lemmatized_keyword)

        # 각 키워드의 빈도를 계산하고 상위 10개 키워드 추출
        keyword_counts = Counter(all_keywords)
        top_10_keywords = keyword_counts.most_common(10)

        # 상위 10개의 키워드를 쉼표로 구분하여 문자열로 변환
        top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]
        keywords_str = ', '.join(top_10_keywords_list)

        # 새로운 데이터프레임 행 생성
        new_row = pd.DataFrame({'title': [csv_title], 'keyword': [keywords_str]})

        # 데이터프레임에 새로운 행 추가 (concat 사용)
        results_df = pd.concat([results_df, new_row], ignore_index=True)

    # 결과를 CSV 파일로 저장
    results_df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# 예시 실행
file_list_csv = 'list_pants.csv'
output_csv = 'output_pants.csv'
stopword_file = '한국어불용어.txt'

process_keywords_from_file_list(file_list_csv, output_csv, stopword_file)

Existing results loaded from output_pants.csv
Trying to load file from: reviews\Deep One Tuck Sweat Shorts [Grey]_reviews.csv
Processing file: reviews\Deep One Tuck Sweat Shorts [Grey]_reviews.csv
Successfully loaded Deep One Tuck Sweat Shorts [Grey]_reviews.csv
Keywords extracted from Deep One Tuck Sweat Shorts [Grey]_reviews.csv
Trying to load file from: reviews\바이오워싱 카펜터 버뮤다 데님 팬츠_라이트블루_reviews.csv
Processing file: reviews\바이오워싱 카펜터 버뮤다 데님 팬츠_라이트블루_reviews.csv
Successfully loaded 바이오워싱 카펜터 버뮤다 데님 팬츠_라이트블루_reviews.csv
Keywords extracted from 바이오워싱 카펜터 버뮤다 데님 팬츠_라이트블루_reviews.csv
Trying to load file from: reviews\데미지 워시드 데님 팬츠-미디엄 블루(Cool Air)_reviews.csv
Processing file: reviews\데미지 워시드 데님 팬츠-미디엄 블루(Cool Air)_reviews.csv
Successfully loaded 데미지 워시드 데님 팬츠-미디엄 블루(Cool Air)_reviews.csv
Keywords extracted from 데미지 워시드 데님 팬츠-미디엄 블루(Cool Air)_reviews.csv
Trying to load file from: reviews\Wide Cargo Half Denim Pants - 5COL_reviews.csv
Processing file: reviews\Wide Cargo Half Denim Pants - 5

In [82]:
df_test = pd.read_csv('output_pants.csv')
df_test = df_test.drop(index=df.index[0]).reset_index(drop=True)

,title,keyword
0,Deep One Tuck Sweat Shorts [Grey]_reviews,"편하다, 사이즈, 바지, 여름, 핏, 기장, 길이, 크다, 허리, 반바지"
1,바이오워싱 카펜터 버뮤다 데님 팬츠_라이트블루_reviews,"사이즈, 허리, 바지, 색감, 기장, 크다, 버뮤다, 적당하다, 핏, 여름"
2,데미지 워시드 데님 팬츠-미디엄 블루(Cool Air)_reviews,"바지, 사이즈, 핏, 크다, 허리, 색감, 여름, 편하다, 기장, 통"
3,Wide Cargo Half Denim Pants - 5COL_reviews,"바지, 사이즈, 허리, 기장, 색감, 편하다, 크다, 길이, 배송, 여름"
4,Deep One Tuck Sweat Shorts [Black]_reviews,"편하다, 바지, 핏, 사이즈, 여름, 기장, 재질, 제로, 적당하다, 반바지"
5,[쿨탠다드] 세미 와이드 히든 밴딩 슬랙스 [블랙]_reviews,"시원하다, 여름, 사이즈, 얇다, 핏, 슬랙스, 편하다, 기장, 와이드, 세미 와이드"
6,버뮤다 와이드 원턱 스웨트 쇼츠 [그레이]_reviews,"편하다, 사이즈, 바지, 여름, 가격, 핏, 적당하다, 시원하다, 재질, 기장"
7,캣 브러쉬 워시드 버뮤다 데님 쇼츠 (SAND BLUE)_reviews,"색감, 사이즈, 허리, 바지, 핏, 버뮤다, 크다, 배송, 기장, 편하다"
8,캣 브러쉬 워시드 버뮤다 데님 쇼츠 (NORMAL BLACK)_reviews,"사이즈, 바지, 허리, 핏, 색감, 워싱, 기장, 배송, 작다, 디자인"
9,Deep One Tuck Sweat Pants [Grey]_reviews,"편하다, 바지, 사이즈, 핏, 기장, 와이드, 크다, 길이, 통, 허리"


In [83]:
df_test.to_csv('output_pants.csv', index=False)

### 여기까지 

In [53]:
file_list_csv = 'list_onepiece.csv'  # 파일명 목록이 담긴 CSV 파일
output_csv = 'output_onepiece.csv'  # 결과를 저장할 CSV 파일
stopword_file = '한국어불용어.txt'  # 불용어 목록 파일

process_keywords_from_file_list(file_list_csv, output_csv, stopword_file)

No existing results. Creating new results file: output_onepiece.csv
Trying to load file from: reviews\reviews_1875663.csv
Processing file: reviews\reviews_1875663.csv
Successfully loaded reviews_1875663.csv
Keywords extracted from reviews_1875663.csv
Trying to load file from: reviews\reviews_3977452.csv
Processing file: reviews\reviews_3977452.csv
Successfully loaded reviews_3977452.csv
Keywords extracted from reviews_3977452.csv
Trying to load file from: reviews\reviews_1875664.csv
Processing file: reviews\reviews_1875664.csv
Successfully loaded reviews_1875664.csv
Keywords extracted from reviews_1875664.csv
Trying to load file from: reviews\reviews_4018731.csv
Processing file: reviews\reviews_4018731.csv
Successfully loaded reviews_4018731.csv
Keywords extracted from reviews_4018731.csv
Trying to load file from: reviews\reviews_3977488.csv
Processing file: reviews\reviews_3977488.csv
Successfully loaded reviews_3977488.csv
Keywords extracted from reviews_3977488.csv
Trying to load fi

In [54]:
df_test = pd.read_csv('output_onepiece.csv')
df_test

,title,keyword
0,reviews_1875663,"길이, 시원하다, 사이즈, 편하다, 비치다, 적당하다, 핏, 여름, 원피스, 라인"
1,reviews_3977452,"짧다, 핏, 길이, 사이즈, 원피스, 소재, 적당하다, 바지, 배송, 재질"
2,reviews_1875664,"시원하다, 길이, 비치다, 편하다, 사이즈, 여름, 짧다, 적당하다, 기장, 핏"
3,reviews_4018731,"바지, 편하다, 짧다, 귀엽다, 길이, 치마, 리본, 디자인, 스커트, 어울리다"
4,reviews_3977488,"짧다, 재질, 흰색, 사이즈, 원피스, 길이, 잡다, 핏, 기장, 비치다"
5,reviews_2551401,"편하다, 바지, 사이즈, 짧다, 허리, 맞다, 길이, 치마, 핏, 크다"
6,reviews_2978106,"편하다, 사이즈, 허리, 시원하다, 크다, 여름, 재질, 단추, 얇다, 소재"
7,reviews_3295891,"색감, 사이즈, 여름, 디자인, 치마, 가격, 하늘색, 밑단, 기장, 색상"
8,reviews_1645762,"사이즈, 허리, 편하다, 길이, 크다, 적당하다, 맞다, 핏, 치마, 만족하다"
9,reviews_1944554,"허리, 사이즈, 길이, 치마, 적당하다, 짧다, 크다, 편하다, 맞다, 배송"


In [55]:
file_list_csv = 'list_jackets.csv'  # 파일명 목록이 담긴 CSV 파일
output_csv = 'output_jackets.csv'  # 결과를 저장할 CSV 파일
stopword_file = '한국어불용어.txt'  # 불용어 목록 파일

process_keywords_from_file_list(file_list_csv, output_csv, stopword_file)

No existing results. Creating new results file: output_jackets.csv
Trying to load file from: reviews\[무료반품]베켄바우어 트랙탑 - 인디고_IP0418.csv
Processing file: reviews\[무료반품]베켄바우어 트랙탑 - 인디고_IP0418.csv
Successfully loaded [무료반품]베켄바우어 트랙탑 - 인디고_IP0418.csv
Keywords extracted from [무료반품]베켄바우어 트랙탑 - 인디고_IP0418.csv
Trying to load file from: reviews\파이어버드 트랙탑 - 블랙_IJ7058.csv
Processing file: reviews\파이어버드 트랙탑 - 블랙_IJ7058.csv
Successfully loaded 파이어버드 트랙탑 - 블랙_IJ7058.csv
Keywords extracted from 파이어버드 트랙탑 - 블랙_IJ7058.csv
Trying to load file from: reviews\ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이.csv
Processing file: reviews\ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이.csv
Successfully loaded ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이.csv
Keywords extracted from ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이.csv
Trying to load file from: reviews\[무료반품]베켄바우어 트랙탑 - 블랙화이트_II5763.csv
Processing file: reviews\[무료반품]베켄바우어 트랙탑 - 블랙화이트_II5763.csv
Successfully loaded [무료반품]베켄바우어 트랙탑 - 블랙화이트_II5763.csv
Keywords extracted from [무료반품]베켄바우어 트랙탑 - 블랙화이트_II5763.csv
Trying to loa

In [56]:
df_test = pd.read_csv('output_jackets.csv')
df_test

,title,keyword
0,[무료반품]베켄바우어 트랙탑 - 인디고_IP0418,"사이즈, 아디다스, 편하다, 색감, 만족하다, 져지, 재질, 기장, 가을, 핏"
1,파이어버드 트랙탑 - 블랙_IJ7058,"사이즈, 배송, 편하다, 아디다스, 핏, 져지, 보리, 크다, 기장, 적당하다"
2,ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이,"색감, 사이즈, 지퍼, 배송, 편하다, 핏, 디자인, 적당하다, 무겁다, 가격"
3,[무료반품]베켄바우어 트랙탑 - 블랙화이트_II5763,"사이즈, 배송, 디자인, 아디다스, 포인트, 핏, 흔하다, 흰색, 보리, 크다"
4,2WAY 스웻 후드 집업 (MELANGE GREY),"사이즈, 지퍼, 편하다, 핏, 적당하다, 후드 집업, 기장, 크다, 길이, 투"
5,[유튜버 PICK]FR-40S 프렌치 워크 5P 자켓_Military Navy,"사이즈, 디자인, 크다, 편하다, 만족하다, 오버핏, 핏, 어울리다, 걸치다, 배송"
6,[쿨탠다드] 릴렉스드 베이식 블레이저 [블랙],"여름, 시원하다, 사이즈, 블 레이저, 얇다, 핏, 편하다, 가격, 가성비, 배송"
7,시어 후드 베스트 블랙,"얇다, 여름, 사이즈, 편하다, 가볍다, 재질, 덥다, 걸치다, 가격, 배송"
8,[SAINT YEAR] 세미 오버 핏 셋업 수트_GREY,"사이즈, 핏, 바지, 가성비, 적당하다, 가격, 결혼식, 얇다, 색감, 셋업"
9,MATIN KIM LOGO COATING JUMPER IN BLACK,"사이즈, 배송, 재질, 디자인, 얇다, 편하다, 마뗑킴, 가을, 만족하다, 핏"


# 일단 접어

In [ ]:
from pykospacing import Spacing

spacing = Spacing()

In [9]:
df = pd.read_csv('[SET] 워셔블 케이블 반팔 니트 세트.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

In [10]:
documents

0      여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...
1                여름에 입 기 딱 좋은 두께에 요 부 드러워서 맨 몸에 입어도 괜찮아요
2      시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...
3      무신사 매장까지 방문해서 재질확인 후 이 구매하였습니다 여름에 입 기 좋은 두께 감...
4       최고에 요 사이즈 살짝 오버핏인데 그냥 너무 좋야 요 제 스펙이랑 비슷하시면 m 사세요
                             ...                        
885                          합리적인 가격이 고 여름 반팔은 역시나 수아레네요
886                     옷 2벌 와 요 선택 하실 때 다른 거로 선택하시길 바래요
887       약간 큰 느낌이 긴 한데 이 가격에 이 정도면 무난 합니다 한 여름에는 힘들 듯해요
888                      사이즈도 좋고 여름에 잘 입고 다니고 있습니다 추천합니다
889                      가성비 젛아요 품은 생각보다 좀 커요 참고하세요 이 뻐요
Name: Review, Length: 890, dtype: object

In [5]:
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 각 문서에서 핵심 키워드 추출
def extract_keywords(text):
    # 각 텍스트에서 상위 5개의 키워드 추출
    keywords = kw_model.extract_keywords(text, vectorizer=vectorizer, top_n=5)
    return keywords

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())

C:\Users\HWAN\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



                                              Review  \
0  여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...   
1               여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요   
2  시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...   
3  무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...   
4     최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요   

                                            Keywords  
0  [(정사이즈로, 0.3535), (구매하시면, 0.3386), (구매하시거나, 0....  
1  [(입어도, 0.6102), (부드러워서, 0.4709), (여름에, 0.4678)...  
2  [(적당히, 0.6256), (라인이, 0.6139), (시원한, 0.6104), ...  
3  [(재질확인후이, 0.7081), (방문해서, 0.6456), (무신사, 0.594...  
4  [(스펙이랑, 0.6513), (사이즈, 0.5276), (사세요, 0.4755),...  


In [6]:
df

,Review,Star,Keywords
0,여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...,5.0,"[(정사이즈로, 0.3535), (구매하시면, 0.3386), (구매하시거나, 0...."
1,여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요,5.0,"[(입어도, 0.6102), (부드러워서, 0.4709), (여름에, 0.4678)..."
2,시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...,5.0,"[(적당히, 0.6256), (라인이, 0.6139), (시원한, 0.6104), ..."
3,무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...,5.0,"[(재질확인후이, 0.7081), (방문해서, 0.6456), (무신사, 0.594..."
4,최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요,5.0,"[(스펙이랑, 0.6513), (사이즈, 0.5276), (사세요, 0.4755),..."
...,...,...,...
885,합리적인 가격이고 여름 반팔은 역시나 수아레네요,5.0,"[(합리적인, 0.6039), (가격이고, 0.5589), (반팔은, 0.4801)..."
886,옷 2벌 와요 선택하실때 다른거로 선택하시길 바래요,5.0,"[(선택하시길, 0.6377), (2벌, 0.521), (다른거로, 0.5048),..."
887,약간 큰 느낌이긴 한데 이 가격에 이정도면 무난 합니다 한여름에는 힘들듯해요,5.0,"[(한여름에는, 0.6743), (무난, 0.6062), (합니다, 0.5799),..."
888,사이즈도 좋고 여름에 잘 입고 다니고 있습니다 추천합니다,5.0,"[(추천합니다, 0.8154), (다니고, 0.6586), (사이즈도, 0.6375..."


In [7]:
from collections import Counter

# 각 문서에서 추출된 키워드를 모은 리스트 생성
all_keywords = []

for keywords in df['Keywords']:
    # keywords는 [(단어, 가중치), (단어, 가중치), ...] 형식이므로 단어만 추출
    all_keywords.extend([keyword[0] for keyword in keywords])

# 키워드의 빈도 계산
keyword_counts = Counter(all_keywords)

# 빈도 상위 10개의 키워드 출력
top_keywords = keyword_counts.most_common(15)

# 상위 10개의 키워드와 그 빈도를 출력
print("상위 15개의 키워드:")
for word, count in top_keywords:
    print(f"{word}: {count}번")

상위 15개의 키워드:
너무: 91번
입기: 60번
가성비: 49번
좋아요: 46번
여름에: 44번
사이즈도: 44번
만족합니다: 41번
색감도: 39번
입고: 35번
생각보다: 34번
사이즈: 30번
마음에: 28번
입을: 27번
재질도: 27번
좋고: 27번


In [8]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입']

# 키워드를 원형으로 변환
def lemmatize_keyword(keyword):
    morphs = okt.pos(keyword, stem=True)  # 형태소 분석 및 어간 추출
    lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']] # 명사, 동사, 형용사 추출
    return ' '.join(lemmatized) if lemmatized else None  # 원형 변환된 단어들을 합침, 없으면 None 반환

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('사이즈', 114), ('색감', 92), ('여름', 72), ('가성 비', 62), ('재질', 59), ('가격', 58), ('배송', 56), ('니트', 55), ('만족하다', 53), ('시원하다', 52)]


In [23]:
csv_title = "[SET] 워셔블 케이블 반팔 니트 세트"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
df = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

In [25]:
df.to_csv('output.csv', index=False)

### 유사도

In [9]:
from collections import defaultdict

# 키워드와 유사도를 저장할 딕셔너리 초기화
keyword_scores = defaultdict(float)

# 키워드를 모두 모아서 유사도를 합산
for keywords in df['Keywords']:
    for keyword, score in keywords:
        keyword_scores[keyword] += score  # 키워드에 유사도(score) 더하기

# 유사도가 높은 상위 10개 키워드 추출
top_10_keywords_with_scores = sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)[:10]

# 결과 출력
print(top_10_keywords_with_scores)


[('너무', 50.934399999999975), ('입기', 29.485300000000002), ('만족합니다', 29.198499999999996), ('가성비', 28.826499999999996), ('사이즈도', 27.499100000000002), ('색감도', 23.14320000000001), ('여름에', 20.394899999999996), ('생각보다', 19.092999999999996), ('입고', 18.4624), ('사이즈', 16.949499999999997)]


In [10]:
from collections import defaultdict
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입', '생각']


# 키워드를 원형으로 변환하는 함수
def lemmatize_keyword(keyword):
    morphs = okt.pos(keyword, stem=True)  # 형태소 분석 + 어간 추출
    # 품사가 명사(Noun)이거나 동사(Verb), 형용사(Adjective)인 경우만 원형으로 변환
    lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']]
    return ' '.join(lemmatized)  # 원형 변환된 단어들을 문자열로 합침

# 키워드와 유사도를 저장할 딕셔너리 초기화
keyword_scores = defaultdict(float)

# 키워드를 모두 모아서 유사도를 합산 (원형으로 변환하여 처리)
for keywords in df['Keywords']:
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)  # 키워드 원형 변환
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 원형이 비어있지 않고 불용어가 아니라면
            keyword_scores[lemmatized_keyword] += score  # 원형 키워드에 유사도 합산

# 유사도가 높은 상위 10개 키워드 추출
top_10_keywords_with_scores = sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)[:10]

# 결과 출력
print(top_10_keywords_with_scores)


[('사이즈', 68.7178), ('색감', 54.8367), ('만족하다', 37.39309999999999), ('가성 비', 36.63450000000001), ('여름', 34.87310000000001), ('재질', 33.6559), ('배송', 32.547399999999996), ('가격', 32.340700000000005), ('시원하다', 30.535999999999994), ('니트', 29.453899999999997)]


# 2

In [36]:
df = pd.read_csv('[2PACK] 쿨 코튼 티셔츠 블랙+화이트.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

In [37]:
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())

                                              Review  \
0  이너티로 입을려고 샀는데 단독으로 입어도 이뻐서 완전 맘에 쏙 후기에 S사이즈도 길...   
1      이너티로도 되게 단단하고 여름에 그냥 이거 하나만 입기에도 시웡하고 이쁜거같아요    
2  옷이 되게 크게 나왔나봐요 평소에 s 아니면 m으로 사는 편인데 다른 후기들 보고 ...   
3                          휘뚜루 마뚜루 입기에는 커버낫 기본이 최고예요   
4  두 번째 주문 편함 재질 부드러움 박음질 튼튼함 흰색은 아주 약간 노란끼 있지만 깔...   

                                            Keywords  
0  [(엉덩이, 0.3746), (완전, 0.3354), (s사이즈도, 0.3219),...  
1  [(이너티로도, 0.658), (시웡하고, 0.6566), (입기에도, 0.6257...  
2  [(아니면, 0.5717), (xs인데도, 0.5459), (늘어날, 0.5391)...  
3  [(기본이, 0.7704), (입기에는, 0.7695), (커버낫, 0.4534),...  
4  [(만족도, 0.58), (흰색은, 0.5688), (박음질, 0.5592), (널...  


In [38]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입']

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('사이즈', 137), ('편하다', 121), ('시원하다', 120), ('재질', 86), ('여름', 74), ('배송', 68), ('티', 56), ('적당하다', 54), ('가격', 52), ('가성 비', 43)]


In [39]:
csv_title = "[2PACK] 쿨 코튼 티셔츠 블랙+화이트"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
new_row = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

df = pd.read_csv('output.csv')
df = pd.concat([df, new_row], ignore_index=True)
df.to_csv('output.csv', index=False)

In [40]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"
1,DOODLE HEART HALF T WHITE GREYISH BLUE,"프린팅, 디자인, 사이즈, 이쁘다, 들다, 편하다, 배송, 예쁘다, 재질, 생각"
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트,"사이즈, 편하다, 시원하다, 재질, 여름, 배송, 티, 적당하다, 가격, 가성 비"


# 리뷰 개별 함수화

In [86]:
# 불용어 처리 함수
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()  # 각 줄마다 불용어를 리스트에 저장
    return stopwords

# 키워드 추출 함수
def extract_keywords(text, kw_model, vectorizer):
    if not text.strip():
        return []  # 빈 텍스트는 건너뛰기
    
    try:
        keywords = kw_model.extract_keywords(text, vectorizer=vectorizer, top_n=10)
        return [keyword for keyword, score in keywords]  # 유사도 점수는 제외하고 키워드만 반환
    except ValueError as e:
        print(f"Skipping document due to error: {e}")
        return []

# 키워드를 원형으로 변환하는 함수
def lemmatize_keyword(keyword, okt, stop_words):
    morphs = okt.pos(keyword, stem=True)  # 형태소 분석 및 어간 추출
    lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective'] and word not in stop_words]
    return ' '.join(lemmatized) if lemmatized else None  # 원형 변환된 단어들을 합침, 없으면 None 반환

# 키워드 추출 및 원형 변환 적용 코드
def process_reviews(file_list_csv, stopword_file):
    # 불용어 파일 불러오기
    stop_words = load_stopwords(stopword_file)

    # 파일명 목록 CSV 불러오기
    file_df = pd.read_csv(file_list_csv)

    # KeyBERT 모델 초기화
    kw_model = KeyBERT()
    vectorizer = CountVectorizer(ngram_range=(1, 1))

    # Okt 형태소 분석기 초기화
    okt = Okt()

    # 파일 처리 시작
    for idx, row in file_df.iterrows():
        file_name = row['File Name']  # 파일명 불러오기
        csv_title = file_name.rsplit('.', 1)[0]  # 파일명에서 확장자 제거하여 제목으로 사용
        file_path = os.path.join('reviews', file_name)  # reviews 폴더 내의 파일 경로 설정

        # 파일 경로 출력
        print(f"Trying to load file from: {file_path}")

        # 데이터 로드 및 정제
        try:
            df = pd.read_csv(file_path)
            print(f"Successfully loaded {file_name}")
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")
            continue

        # 리뷰 데이터 정제
        df = clean_review_column(df, 'Review')
        documents = df['Review']

        # 전체 문서에 대해 핵심 키워드 추출
        df['pre_keywords'] = documents.apply(lambda text: extract_keywords(text, kw_model, vectorizer))

        # 원형으로 변환된 키워드를 새로운 열에 추가
        df['keyword'] = df['pre_keywords'].apply(
            lambda keywords: ', '.join([lemmatize_keyword(keyword, okt, stop_words) for keyword in keywords if lemmatize_keyword(keyword, okt, stop_words)])
        )

        # 리뷰와 키워드 열만 따로 저장
        output_df = df[['Review', 'keyword']]
        output_file = f'{csv_title}_2.csv'
        output_df.to_csv(output_file, index=False)
        print(f"File saved to {output_file}")

In [89]:
process_reviews('list_onepiece.csv', '한국어불용어.txt')

Trying to load file from: reviews\reviews_1875663.csv
Successfully loaded reviews_1875663.csv
File saved to reviews_1875663_2.csv
Trying to load file from: reviews\reviews_3977452.csv
Successfully loaded reviews_3977452.csv
File saved to reviews_3977452_2.csv
Trying to load file from: reviews\reviews_1875664.csv
Successfully loaded reviews_1875664.csv
File saved to reviews_1875664_2.csv
Trying to load file from: reviews\reviews_4018731.csv
Successfully loaded reviews_4018731.csv
File saved to reviews_4018731_2.csv
Trying to load file from: reviews\reviews_3977488.csv
Successfully loaded reviews_3977488.csv
File saved to reviews_3977488_2.csv
Trying to load file from: reviews\reviews_2551401.csv
Successfully loaded reviews_2551401.csv
File saved to reviews_2551401_2.csv
Trying to load file from: reviews\reviews_2978106.csv
Successfully loaded reviews_2978106.csv
File saved to reviews_2978106_2.csv
Trying to load file from: reviews\reviews_3295891.csv
Successfully loaded reviews_3295891.

In [90]:
process_reviews('list_tshirts.csv', '한국어불용어.txt')

Trying to load file from: reviews\DOODLE HEART HALF T WHITE GREYISH BLUE.csv
Successfully loaded DOODLE HEART HALF T WHITE GREYISH BLUE.csv
File saved to DOODLE HEART HALF T WHITE GREYISH BLUE_2.csv
Trying to load file from: reviews\[SET] 워셔블 케이블 반팔 니트 세트.csv
Successfully loaded [SET] 워셔블 케이블 반팔 니트 세트.csv
File saved to [SET] 워셔블 케이블 반팔 니트 세트_2.csv
Trying to load file from: reviews\[2PACK] 쿨 코튼 티셔츠 블랙+화이트.csv
Successfully loaded [2PACK] 쿨 코튼 티셔츠 블랙+화이트.csv
File saved to [2PACK] 쿨 코튼 티셔츠 블랙+화이트_2.csv
Trying to load file from: reviews\[3천원 결제혜택]링클 체크 박시 오버핏 롤업 하프 셔츠 다크 네이비.csv
Successfully loaded [3천원 결제혜택]링클 체크 박시 오버핏 롤업 하프 셔츠 다크 네이비.csv
File saved to [3천원 결제혜택]링클 체크 박시 오버핏 롤업 하프 셔츠 다크 네이비_2.csv
Trying to load file from: reviews\460G 컷 헤비 피그먼트 티셔츠-차콜-.csv
Successfully loaded 460G 컷 헤비 피그먼트 티셔츠-차콜-.csv
File saved to 460G 컷 헤비 피그먼트 티셔츠-차콜-_2.csv
Trying to load file from: reviews\[16수 코마사] 2PACK SMALL ARCH T-SHIRT WHITE _ BLACK.csv
Successfully loaded [16수 코마사] 2PACK SMALL ARCH T-SHIRT WHIT

# 상품 전체에 대한

In [31]:
df = pd.read_csv('DOODLE HEART HALF T WHITE GREYISH BLUE.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

In [6]:
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())

KeyboardInterrupt: 

In [33]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입']

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('프린팅', 157), ('디자인', 138), ('사이즈', 104), ('이쁘다', 94), ('들다', 57), ('편하다', 56), ('배송', 51), ('예쁘다', 51), ('재질', 49), ('생각', 45)]


In [34]:
csv_title = "DOODLE HEART HALF T WHITE GREYISH BLUE"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
new_row = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

df = pd.read_csv('output.csv')
df = pd.concat([df, new_row], ignore_index=True)
df.to_csv('output.csv', index=False)

In [35]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"
1,DOODLE HEART HALF T WHITE GREYISH BLUE,"프린팅, 디자인, 사이즈, 이쁘다, 들다, 편하다, 배송, 예쁘다, 재질, 생각"


### 함수화 

In [77]:
# 메인 함수
def process_keywords_from_csv(file_name, output_csv, csv_title):
    # 파일 경로 확인
    if not os.path.isfile(file_name):
        print(f"Error: '{file_name}' 경로에 파일이 존재하지 않습니다.")
        return
    
    # 데이터 로드 및 정제
    df = pd.read_csv(file_name)
    df = clean_review_column(df, 'Review')
    documents = df['Review']

    # KeyBERT 모델 초기화
    kw_model = KeyBERT()
    vectorizer = CountVectorizer(ngram_range=(1, 1))

    # 전체 문서에 대해 핵심 키워드 추출
    df['Keywords'] = documents.apply(lambda doc: extract_keywords(doc, kw_model, vectorizer))

    # Okt 형태소 분석기 초기화 및 불용어 로드
    okt = Okt()
    stop_words = load_stopwords('한국어불용어.txt')  # 사용자 지정 불용어 파일

    # 키워드를 모두 모아서 리스트로 풀기
    all_keywords = []
    for keywords in df['Keywords']:
        for keyword in keywords:
            lemmatized_keyword = lemmatize_keyword(keyword, okt, stop_words)
            if lemmatized_keyword:  # 불용어 필터링 후 추가
                all_keywords.append(lemmatized_keyword)

    # 각 키워드의 빈도를 계산하고 상위 10개 키워드 추출
    keyword_counts = Counter(all_keywords)
    top_10_keywords = keyword_counts.most_common(10)

    # 상위 10개의 키워드를 쉼표로 구분하여 문자열로 변환
    top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]
    keywords_str = ', '.join(top_10_keywords_list)

    # 새로운 데이터프레임 행 추가
    new_row = pd.DataFrame({
        'title': [csv_title],
        'keyword': [keywords_str]
    })

    # 기존 CSV 파일에 새로운 행 추가
    if os.path.isfile(output_csv):
        df_output = pd.read_csv(output_csv)
    else:
        df_output = pd.DataFrame(columns=['title', 'keyword'])  # 파일이 없으면 새로 생성
    
    df_output = pd.concat([df_output, new_row], ignore_index=True)
    df_output.to_csv(output_csv, index=False)

In [78]:
# 사용 예시
# 아래 file_name, output_csv, csv_title 부분만 수정해서 사용하세요.
file_name = 'reviews/DOTSSB.csv'
output_csv = 'output_pants.csv'
csv_title = 'Deep One Tuck Sweat Shorts [Black]'

process_keywords_from_csv(file_name, output_csv, csv_title)

In [79]:
df_output = pd.read_csv('output_pants.csv')
df_output

,title,keyword
0,Deep One Tuck Sweat Shorts [Black],"편하다, 바지, 핏, 사이즈, 여름, 기장, 재질, 제로, 적당하다, 반바지"


# 상품 내 각 리뷰에 대한

In [31]:
df = pd.read_csv('1967 JET BLACK JEANS [WIDE STRAIGHT]_reviews.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
#print(df[['Review', 'Keywords']].head())
df

,Review,Star,Keywords
0,다리가 상당히 길어보입니다 가을에 이것만 땡겨입겠습 둥 아 밑위가 좀 짧아요,5.0,"[상당히, 길어보입니다, 다리가, 가을에, 짧아요, 이것만, 밑위가, 땡겨입겠습]"
1,아우 예뻐용 홍 말라서 이랑 고민 많이 됐는데 입어도 예뿌네용,5.0,"[입어도, 이랑, 말라서, 고민, 아우, 예뿌네용, 예뻐용, 많이, 됐는데]"
2,여자후기 없어서 고민 많이 하다가 그냥 샀는데 핏이랑 색감 다 맘에 드네요 길다고 ...,5.0,"[롤업하는, 다니는, 여자후기, 길다고, 색감, 맘에, 이러고, 하다가, 드네요, 고민]"
3,사이즈는 정사이즈 가시면 될 것 같아요 일직선으로 떨어지는 스트레이트 핏이 이쁩니다...,5.0,"[일직선으로, 정사이즈, 워싱은, 스트레이트, 가시면, 사이즈는, 핏이, 이쁩니다,..."
4,튼실한 제 빵댕이도 견딜줄 몰랐습니다 강추합니다,5.0,"[강추합니다, 튼실한, 견딜줄, 빵댕이도, 몰랐습니다]"
...,...,...,...
995,와이드팬츠 있었는데 버리고 브랜디드 와이드팬츠 삿는데 이쁘네요 허리 정도 입는데 도...,5.0,"[와이드팬츠, 추천합니다, 입는데, 브랜디드, 버리고, 허리, 정도, 있었는데, 이..."
996,이쁘네요 브랜디드 청바지 맛집입니다 많이 파세요,5.0,"[청바지, 파세요, 브랜디드, 이쁘네요, 맛집입니다, 많이]"
997,핏 미치고 색감 미치고 적당히 와이드해서 너무 예뻐요 편하기도 짱좋아여,5.0,"[미치고, 와이드해서, 편하기도, 적당히, 색감, 너무, 짱좋아여, 예뻐요]"
998,믿고 사는 부랜디드 입니다 바지 색감 핏가 딱 원하던 예쁜옷이에용,5.0,"[원하던, 입니다, 부랜디드, 사는, 색감, 바지, 핏가, 예쁜옷이에용, 믿고]"


In [36]:
stop_words = ['좋다', '입다', '같다', '이다', '있다', '입', '생각', '하다', '기다', '들다', '안', '브랜디드', '예쁘다', '이쁘다', '옷', '늘', '용']


# 원형으로 변환된 키워드를 새로운 열에 추가
df['Lemmatized_Keywords'] = df['Keywords'].apply(
    lambda keywords: ', '.join([lemmatize_keyword(keyword) for keyword in keywords if lemmatize_keyword(keyword)])
)
df

,Review,Star,Keywords,Lemmatized_Keywords
0,다리가 상당히 길어보입니다 가을에 이것만 땡겨입겠습 둥 아 밑위가 좀 짧아요,5.0,"[상당히, 길어보입니다, 다리가, 가을에, 짧아요, 이것만, 밑위가, 땡겨입겠습]","상당하다, 길다 보이다, 다리, 가을, 짧다, 것, 밑위, 땡기다 습"
1,아우 예뻐용 홍 말라서 이랑 고민 많이 됐는데 입어도 예뿌네용,5.0,"[입어도, 이랑, 말라서, 고민, 아우, 예뿌네용, 예뻐용, 많이, 됐는데]","말, 고민, 아우, 뿌, 돼다"
2,여자후기 없어서 고민 많이 하다가 그냥 샀는데 핏이랑 색감 다 맘에 드네요 길다고 ...,5.0,"[롤업하는, 다니는, 여자후기, 길다고, 색감, 맘에, 이러고, 하다가, 드네요, 고민]","롤 업다, 다니다, 여자 후기, 색감, 맘, 이르다, 드네, 고민"
3,사이즈는 정사이즈 가시면 될 것 같아요 일직선으로 떨어지는 스트레이트 핏이 이쁩니다...,5.0,"[일직선으로, 정사이즈, 워싱은, 스트레이트, 가시면, 사이즈는, 핏이, 이쁩니다,...","일직선, 정사 이즈, 워싱, 스트레이트, 가시, 사이즈, 핏, 떨어지다, 낫다"
4,튼실한 제 빵댕이도 견딜줄 몰랐습니다 강추합니다,5.0,"[강추합니다, 튼실한, 견딜줄, 빵댕이도, 몰랐습니다]","강추, 튼실하다, 견디다, 빵댕, 모르다"
...,...,...,...,...
995,와이드팬츠 있었는데 버리고 브랜디드 와이드팬츠 삿는데 이쁘네요 허리 정도 입는데 도...,5.0,"[와이드팬츠, 추천합니다, 입는데, 브랜디드, 버리고, 허리, 정도, 있었는데, 이...","와이드 팬츠, 추천, 브랜디 드, 버리다, 허리, 정도, 삿는데"
996,이쁘네요 브랜디드 청바지 맛집입니다 많이 파세요,5.0,"[청바지, 파세요, 브랜디드, 이쁘네요, 맛집입니다, 많이]","청바지, 파다, 브랜디 드, 맛집"
997,핏 미치고 색감 미치고 적당히 와이드해서 너무 예뻐요 편하기도 짱좋아여,5.0,"[미치고, 와이드해서, 편하기도, 적당히, 색감, 너무, 짱좋아여, 예뻐요]","미치다, 와이드, 편하다, 적당하다, 색감"
998,믿고 사는 부랜디드 입니다 바지 색감 핏가 딱 원하던 예쁜옷이에용,5.0,"[원하던, 입니다, 부랜디드, 사는, 색감, 바지, 핏가, 예쁜옷이에용, 믿고]","원하다, 부랜디드, 살다, 색감, 바지, 핏, 믿다"


In [35]:
df['Review'][3]

'사이즈는 정사이즈 가시면 될 것 같아요 일직선으로 떨어지는 스트레이트 핏이 이쁩니다 워싱은 이 나은거같아요'

In [71]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입', '생각', '하다', '기다', '들다']

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('기장', 124), ('바지', 118), ('사이즈', 112), ('색감', 80), ('만족하다', 68), ('적당하다', 59), ('브랜디 드', 58), ('핏', 57), ('이쁘다', 56), ('허리', 55)]


In [72]:
csv_title = "1967 JET BLACK JEANS [WIDE STRAIGHT]_reviews"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
new_row = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

df = pd.read_csv('output.csv')
df = pd.concat([df, new_row], ignore_index=True)
df.to_csv('output.csv', index=False)

In [73]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"
1,DOODLE HEART HALF T WHITE GREYISH BLUE,"프린팅, 디자인, 사이즈, 이쁘다, 들다, 편하다, 배송, 예쁘다, 재질, 생각"
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트,"사이즈, 편하다, 시원하다, 재질, 여름, 배송, 티, 적당하다, 가격, 가성 비"
3,아우터3. ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이,"색감, 사이즈, 배송, 생각, 지퍼, 편하다, 디자인, 이쁘다, 적당하다, 무겁다"
4,아우터2. 파이어버드 트랙탑 - 블랙 IJ7058,"사이즈, 편하다, 배송, 아디다스, 적당하다, 이쁘다, 만족하다, 크다, 져지, 디자인"
5,42283 루즈핏 워크쇼츠 Black_reviews.csv,"사이즈, 허리, 디 키즈, 재질, 바지, 크다, 만족하다, 업 하다, 기장, 핏"
6,(여름버전추가) 투턱 와이드 롱 스웨트 팬츠 [그레이]_reviews,"편하다, 여름, 기장, 바지, 배송, 사이즈, 핏, 시원하다, 이쁘다, 만족하다"
7,아우터1. 베켄바우어 트랙탑 - 인디고 IP0418,"사이즈, 아디다스, 편하다, 색감, 예쁘다, 만족하다, 이쁘다, 실물, 역시, 기장"
8,1967 JET BLACK JEANS [WIDE STRAIGHT]_reviews,"기장, 바지, 사이즈, 색감, 만족하다, 적당하다, 브랜디 드, 핏, 이쁘다, 허리"


# 10

In [74]:
df = pd.read_csv('reviews_1875663.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())

                                              Review  \
0  XS사려다 S샀는데 XS살걸 우선 다른 피케원피스보다 기장이 길었고 시원한 재질이여...   
1  키가 매우매우 크기 때무네 저에게는 미니기장이긴 한데 짧을건 이미 예상하고 구매했습...   
2  네이비랑 블랙 고밈하다가 블랙구매했어요 허리 잘록하게 잘들어가고 핏이뻐요 다만 많이...   
3      처음에 s살까 xs살까 고민하다가 xs샀어요 질도 좋고 예뻐요 네이비도 살까봐여    
4  많이 짧지도 않고 깔끔한 핏에 허리라인이 살짝 들어가서 예뻐요 옷감이 얇아서 밝은데...   

                                            Keywords  
0  [(xs살걸, 0.7127), (xs사려다, 0.6514), (재질이여서, 0.56...  
1  [(갠춘합니당, 0.3518), (미니기장이긴, 0.3505), (속치마, 0.30...  
2  [(네이비랑, 0.5784), (잘들어가고, 0.5773), (잘록하게, 0.567...  
3  [(네이비도, 0.752), (고민하다가, 0.7416), (처음에, 0.6394)...  
4  [(비쳐보여서, 0.7145), (허리라인이, 0.6891), (속치마를, 0.65...  


In [75]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입', '생각', '하다', '기다', '들다']

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('시원하다', 167), ('길이', 125), ('사이즈', 123), ('편하다', 115), ('적당하다', 98), ('비치다', 83), ('원피스', 70), ('배송', 53), ('재질', 51), ('라인', 51)]


In [76]:
csv_title = "reviews_1875663"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
new_row = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

df = pd.read_csv('output.csv')
df = pd.concat([df, new_row], ignore_index=True)
df.to_csv('output.csv', index=False)

In [77]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"
1,DOODLE HEART HALF T WHITE GREYISH BLUE,"프린팅, 디자인, 사이즈, 이쁘다, 들다, 편하다, 배송, 예쁘다, 재질, 생각"
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트,"사이즈, 편하다, 시원하다, 재질, 여름, 배송, 티, 적당하다, 가격, 가성 비"
3,아우터3. ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이,"색감, 사이즈, 배송, 생각, 지퍼, 편하다, 디자인, 이쁘다, 적당하다, 무겁다"
4,아우터2. 파이어버드 트랙탑 - 블랙 IJ7058,"사이즈, 편하다, 배송, 아디다스, 적당하다, 이쁘다, 만족하다, 크다, 져지, 디자인"
5,42283 루즈핏 워크쇼츠 Black_reviews.csv,"사이즈, 허리, 디 키즈, 재질, 바지, 크다, 만족하다, 업 하다, 기장, 핏"
6,(여름버전추가) 투턱 와이드 롱 스웨트 팬츠 [그레이]_reviews,"편하다, 여름, 기장, 바지, 배송, 사이즈, 핏, 시원하다, 이쁘다, 만족하다"
7,아우터1. 베켄바우어 트랙탑 - 인디고 IP0418,"사이즈, 아디다스, 편하다, 색감, 예쁘다, 만족하다, 이쁘다, 실물, 역시, 기장"
8,1967 JET BLACK JEANS [WIDE STRAIGHT]_reviews,"기장, 바지, 사이즈, 색감, 만족하다, 적당하다, 브랜디 드, 핏, 이쁘다, 허리"
9,reviews_1875663,"시원하다, 길이, 사이즈, 편하다, 적당하다, 비치다, 원피스, 배송, 재질, 라인"


# 11

In [78]:
df = pd.read_csv('reviews_1944554.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())

                                              Review  \
0                     이뻐요 최고 꾸미고 나갈 때 진짜 자주 입음 무난꾸안꾸   
1  딸아이가 맘에들어합니다 160초반에 48킬로 딱맞습니다 기본디자인이고 만듬세 좋아요...   
2          생각보다 짧아서 당황했지만 두께감 적당하고 허리 들뜸 없어서 너무 좋아요    
3              허리는 맞는데 골반이 너무 껴요 길이는 딱 좋은데 입고벗기 불편해요   
4           오랜만에 짧은청치마 원하던스타일이에요 날씨가 따뜻해서 외출복으로 픽했어요   

                                            Keywords  
0  [(입음, 0.7701), (나갈, 0.6841), (자주, 0.6678), (최고...  
1  [(기본디자인이고, 0.5646), (맘에들어합니다, 0.5142), (160초반에...  
2  [(적당하고, 0.7706), (생각보다, 0.7002), (너무, 0.572), ...  
3  [(허리는, 0.6768), (불편해요, 0.6754), (골반이, 0.668), ...  
4  [(원하던스타일이에요, 0.8043), (날씨가, 0.5828), (오랜만에, 0....  


In [79]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입', '생각', '하다', '기다', '들다']

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('사이즈', 169), ('허리', 157), ('적당하다', 121), ('길이', 106), ('치마', 69), ('편하다', 63), ('배송', 63), ('크다', 56), ('만족하다', 47), ('색상', 44)]


In [80]:
csv_title = "reviews_1944554"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
new_row = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

df = pd.read_csv('output.csv')
df = pd.concat([df, new_row], ignore_index=True)
df.to_csv('output.csv', index=False)

In [81]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"
1,DOODLE HEART HALF T WHITE GREYISH BLUE,"프린팅, 디자인, 사이즈, 이쁘다, 들다, 편하다, 배송, 예쁘다, 재질, 생각"
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트,"사이즈, 편하다, 시원하다, 재질, 여름, 배송, 티, 적당하다, 가격, 가성 비"
3,아우터3. ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이,"색감, 사이즈, 배송, 생각, 지퍼, 편하다, 디자인, 이쁘다, 적당하다, 무겁다"
4,아우터2. 파이어버드 트랙탑 - 블랙 IJ7058,"사이즈, 편하다, 배송, 아디다스, 적당하다, 이쁘다, 만족하다, 크다, 져지, 디자인"
5,42283 루즈핏 워크쇼츠 Black_reviews.csv,"사이즈, 허리, 디 키즈, 재질, 바지, 크다, 만족하다, 업 하다, 기장, 핏"
6,(여름버전추가) 투턱 와이드 롱 스웨트 팬츠 [그레이]_reviews,"편하다, 여름, 기장, 바지, 배송, 사이즈, 핏, 시원하다, 이쁘다, 만족하다"
7,아우터1. 베켄바우어 트랙탑 - 인디고 IP0418,"사이즈, 아디다스, 편하다, 색감, 예쁘다, 만족하다, 이쁘다, 실물, 역시, 기장"
8,1967 JET BLACK JEANS [WIDE STRAIGHT]_reviews,"기장, 바지, 사이즈, 색감, 만족하다, 적당하다, 브랜디 드, 핏, 이쁘다, 허리"
9,reviews_1875663,"시원하다, 길이, 사이즈, 편하다, 적당하다, 비치다, 원피스, 배송, 재질, 라인"


# 12

In [87]:
df = pd.read_csv('reviews_2551401.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())

                                              Review  \
0  168 기준 살짝 짧은 기장이기는 하지만 속바지가 있어서 편하게 입기 좋아요 옆쪽 ...   
1  디자인은 정말 예쁜데 안에 박음질은 엉망이에요 단 기장이 아예 다르게 어긋나있는데도...   
2              교복 치마 대신 입으려고 샀는데 편하고 핏도 예쁘고 너무 만족합니다   
3  저렴한 가격인데 품질 좋네요 안에 속바지가 있어서 좀 짧긴해도 부담없어요 저한테는 ...   
4  사이즈랑 길이가 완전 딱입니다 160대초반 44사이즈 입으시면 xs사이즈 예쁘게 잘...   

                                            Keywords  
0  [(기장이기는, 0.6006), (심심하지, 0.5184), (속바지가, 0.490...  
1  [(엉망이에요, 0.697), (허접할거, 0.6658), (기장이, 0.6189)...  
2  [(만족합니다, 0.7323), (입으려고, 0.6412), (편하고, 0.6041...  
3  [(엉덩이가, 0.667), (조여서, 0.6089), (저렴한, 0.5945), ...  
4  [(xs사이즈, 0.5646), (44사이즈, 0.5194), (입으시면, 0.42...  


In [88]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입', '생각', '하다', '기다', '들다', '안']

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('편하다', 253), ('바지', 186), ('사이즈', 175), ('허리', 118), ('길이', 61), ('예쁘다', 56), ('만족하다', 53), ('엉덩이', 53), ('적당하다', 52), ('기장', 49)]


In [94]:
csv_title = "reviews_2551401"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
new_row = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

df = pd.read_csv('output.csv')
df = pd.concat([df, new_row], ignore_index=True)
df.to_csv('output.csv', index=False)

In [95]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"
1,DOODLE HEART HALF T WHITE GREYISH BLUE,"프린팅, 디자인, 사이즈, 이쁘다, 들다, 편하다, 배송, 예쁘다, 재질, 생각"
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트,"사이즈, 편하다, 시원하다, 재질, 여름, 배송, 티, 적당하다, 가격, 가성 비"
3,아우터3. ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이,"색감, 사이즈, 배송, 생각, 지퍼, 편하다, 디자인, 이쁘다, 적당하다, 무겁다"
4,아우터2. 파이어버드 트랙탑 - 블랙 IJ7058,"사이즈, 편하다, 배송, 아디다스, 적당하다, 이쁘다, 만족하다, 크다, 져지, 디자인"
5,42283 루즈핏 워크쇼츠 Black_reviews.csv,"사이즈, 허리, 디 키즈, 재질, 바지, 크다, 만족하다, 업 하다, 기장, 핏"
6,(여름버전추가) 투턱 와이드 롱 스웨트 팬츠 [그레이]_reviews,"편하다, 여름, 기장, 바지, 배송, 사이즈, 핏, 시원하다, 이쁘다, 만족하다"
7,아우터1. 베켄바우어 트랙탑 - 인디고 IP0418,"사이즈, 아디다스, 편하다, 색감, 예쁘다, 만족하다, 이쁘다, 실물, 역시, 기장"
8,1967 JET BLACK JEANS [WIDE STRAIGHT]_reviews,"기장, 바지, 사이즈, 색감, 만족하다, 적당하다, 브랜디 드, 핏, 이쁘다, 허리"
9,reviews_1875663,"시원하다, 길이, 사이즈, 편하다, 적당하다, 비치다, 원피스, 배송, 재질, 라인"


In [99]:
df = df.drop(index=12)

In [101]:
df = df.reset_index(drop=True)
df.to_csv('output.csv', index=False)

In [102]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"
1,DOODLE HEART HALF T WHITE GREYISH BLUE,"프린팅, 디자인, 사이즈, 이쁘다, 들다, 편하다, 배송, 예쁘다, 재질, 생각"
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트,"사이즈, 편하다, 시원하다, 재질, 여름, 배송, 티, 적당하다, 가격, 가성 비"
3,아우터3. ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이,"색감, 사이즈, 배송, 생각, 지퍼, 편하다, 디자인, 이쁘다, 적당하다, 무겁다"
4,아우터2. 파이어버드 트랙탑 - 블랙 IJ7058,"사이즈, 편하다, 배송, 아디다스, 적당하다, 이쁘다, 만족하다, 크다, 져지, 디자인"
5,42283 루즈핏 워크쇼츠 Black_reviews.csv,"사이즈, 허리, 디 키즈, 재질, 바지, 크다, 만족하다, 업 하다, 기장, 핏"
6,(여름버전추가) 투턱 와이드 롱 스웨트 팬츠 [그레이]_reviews,"편하다, 여름, 기장, 바지, 배송, 사이즈, 핏, 시원하다, 이쁘다, 만족하다"
7,아우터1. 베켄바우어 트랙탑 - 인디고 IP0418,"사이즈, 아디다스, 편하다, 색감, 예쁘다, 만족하다, 이쁘다, 실물, 역시, 기장"
8,1967 JET BLACK JEANS [WIDE STRAIGHT]_reviews,"기장, 바지, 사이즈, 색감, 만족하다, 적당하다, 브랜디 드, 핏, 이쁘다, 허리"
9,reviews_1875663,"시원하다, 길이, 사이즈, 편하다, 적당하다, 비치다, 원피스, 배송, 재질, 라인"


# 기타(파일목록 정리)

In [1]:
import os
import pandas as pd

# 현재 디렉토리 경로 설정
current_directory = os.getcwd()

# 'reviews' 폴더 경로 설정
reviews_folder = os.path.join(current_directory, 'preprocessing_reviews')

# 'reviews' 폴더에 있는 파일 목록을 읽어옴
file_list = os.listdir(reviews_folder)

# 파일 목록을 데이터프레임으로 변환
df = pd.DataFrame(file_list, columns=['File Name'])

# 데이터프레임 출력
df

,File Name
0,1645762.csv
1,1875663.csv
2,1875664.csv
3,1944554.csv
4,2551401.csv
5,2978106.csv
6,2WAY 스웻 후드 집업 (MELANGE GREY).csv
7,3295891.csv
8,3977452.csv
9,3977488.csv


In [2]:

# 새롭게 정렬할 순서
new_order = [
    "DOODLE HEART HALF T WHITE GREYISH BLUE.csv",
    "[SET] 워셔블 케이블 반팔 니트 세트.csv",
    "[2PACK] 쿨 코튼 티셔츠 블랙+화이트.csv",
    "[3천원 결제혜택]링클 체크 박시 오버핏 롤업 하프 셔츠 다크 네이비.csv",
    "460G 컷 헤비 피그먼트 티셔츠-차콜-.csv",
    "[16수 코마사] 2PACK SMALL ARCH T-SHIRT WHITE _ BLACK.csv",
    "빈티지 워싱 네이비 체크셔츠.csv",
    "[3PACK] 에어쿨링 드라이 기능성 무지 반팔티.csv",
    "TAG OG TEE - WHITE.csv",
    "[2PACK] EL 스트리트 아트워크 오버핏 반팔티 2종 2PACK.csv",
    "1875663.csv",
    "3977452.csv",
    "1875664.csv",
    "4018731.csv",
    "3977488.csv",
    "2551401.csv",
    "2978106.csv",
    "3295891.csv",
    "1645762.csv",
    "1944554.csv",
    "Deep One Tuck Sweat Shorts [Grey].csv",
    "바이오워싱 카펜터 버뮤다 데님 팬츠_라이트블루.csv",
    "데미지 워시드 데님 팬츠-미디엄 블루(Cool Air).csv",
    "Wide Cargo Half Denim Pants - 5COL.csv",
    "Deep One Tuck Sweat Shorts [Black].csv",
    "[쿨탠다드] 세미 와이드 히든 밴딩 슬랙스 [블랙].csv",
    "버뮤다 와이드 원턱 스웨트 쇼츠 [그레이].csv",
    "캣 브러쉬 워시드 버뮤다 데님 쇼츠 (SAND BLUE).csv",
    "캣 브러쉬 워시드 버뮤다 데님 쇼츠 (NORMAL BLACK).csv",
    "Deep One Tuck Sweat Pants [Grey].csv",
    "[무료반품]베켄바우어 트랙탑 - 인디고_IP0418.csv",
    "파이어버드 트랙탑 - 블랙_IJ7058.csv",
    "ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이.csv",
    "[무료반품]베켄바우어 트랙탑 - 블랙화이트_II5763.csv",
    "2WAY 스웻 후드 집업 (MELANGE GREY).csv",
    "[유튜버 PICK]FR-40S 프렌치 워크 5P 자켓_Military Navy.csv",
    "[쿨탠다드] 릴렉스드 베이식 블레이저 [블랙].csv",
    "시어 후드 베스트 블랙.csv",
    "[SAINT YEAR] 세미 오버 핏 셋업 수트_GREY.csv",
    "MATIN KIM LOGO COATING JUMPER IN BLACK.csv"
]

# 기존 리스트와 새 순서를 이용해 데이터프레임 재정렬
df = pd.DataFrame(file_list, columns=['File Name'])

# 새로운 순서로 정렬된 데이터프레임 생성
df_sorted = pd.DataFrame(new_order, columns=['File Name'])

# 데이터프레임 출력
df_sorted

,File Name
0,DOODLE HEART HALF T WHITE GREYISH BLUE.csv
1,[SET] 워셔블 케이블 반팔 니트 세트.csv
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트.csv
3,[3천원 결제혜택]링클 체크 박시 오버핏 롤업 하프 셔츠 다크 네이비.csv
4,460G 컷 헤비 피그먼트 티셔츠-차콜-.csv
5,[16수 코마사] 2PACK SMALL ARCH T-SHIRT WHITE _ BLA...
6,빈티지 워싱 네이비 체크셔츠.csv
7,[3PACK] 에어쿨링 드라이 기능성 무지 반팔티.csv
8,TAG OG TEE - WHITE.csv
9,[2PACK] EL 스트리트 아트워크 오버핏 반팔티 2종 2PACK.csv


In [3]:
# 10개씩 끊어서 데이터프레임으로 만들고 동적으로 변수 이름 지정
for i in range(0, len(new_order), 10):
    df_name = f"df_{i//10 + 1}"
    vars()[df_name] = pd.DataFrame(new_order[i:i+10], columns=['File Name'])

In [4]:
df_1
df_1.to_csv("list_tshirts.csv", index=False)

In [5]:
df_2
df_2.to_csv("list_onepiece.csv", index=False)

In [6]:
df_3
df_3.to_csv("list_pants.csv", index=False)

In [7]:
df_4
df_4.to_csv("list_jackets.csv", index=False)

In [8]:
# 파일 열기 검증 함수
def check_file_open(file_path):
    try:
        # 파일을 판다스로 열 수 있는지 검증 (csv 파일로 가정)
        pd.read_csv(file_path)
        return True
    except Exception as e:
        print(f"Error opening {file_path}: {e}")
        return False

# 파일 열기 검증 실행
for file_name in df_1['File Name']:
    file_path = os.path.join(reviews_folder, file_name)
    if check_file_open(file_path):
        print(f"{file_name} 파일이 정상적으로 열렸습니다.")
    else:
        print(f"{file_name} 파일을 여는 데 실패했습니다.")

DOODLE HEART HALF T WHITE GREYISH BLUE.csv 파일이 정상적으로 열렸습니다.
[SET] 워셔블 케이블 반팔 니트 세트.csv 파일이 정상적으로 열렸습니다.
[2PACK] 쿨 코튼 티셔츠 블랙+화이트.csv 파일이 정상적으로 열렸습니다.
[3천원 결제혜택]링클 체크 박시 오버핏 롤업 하프 셔츠 다크 네이비.csv 파일이 정상적으로 열렸습니다.
460G 컷 헤비 피그먼트 티셔츠-차콜-.csv 파일이 정상적으로 열렸습니다.
[16수 코마사] 2PACK SMALL ARCH T-SHIRT WHITE _ BLACK.csv 파일이 정상적으로 열렸습니다.
빈티지 워싱 네이비 체크셔츠.csv 파일이 정상적으로 열렸습니다.
[3PACK] 에어쿨링 드라이 기능성 무지 반팔티.csv 파일이 정상적으로 열렸습니다.
TAG OG TEE - WHITE.csv 파일이 정상적으로 열렸습니다.
[2PACK] EL 스트리트 아트워크 오버핏 반팔티 2종 2PACK.csv 파일이 정상적으로 열렸습니다.


In [3]:
# 이름 비교해서 목록에 맞춰서 파일명 업데이트
import os
import pandas as pd

def rename_files_based_on_list(file_list_csv, folder_path):
    # 파일명 목록 CSV 불러오기
    file_df = pd.read_csv(file_list_csv)

    # 폴더 내 실제 파일 목록 불러오기
    actual_files = os.listdir(folder_path)

    # 파일 처리 시작
    for idx, row in file_df.iterrows():
        correct_name = row['File Name']  # 목록에 있는 올바른 파일명
        
        # 가장 유사한 실제 파일 찾기
        for actual_file in actual_files:
            # 파일명을 확장자 포함해서 비교
            actual_name, actual_ext = os.path.splitext(actual_file)
            correct_name_no_ext, correct_ext = os.path.splitext(correct_name)
            
            # 대소문자 무시하고 유사도 비교 (확장자가 일치하면)
            if actual_name.lower() == correct_name_no_ext.lower() and actual_ext == correct_ext:
                # 파일 경로 설정
                old_file_path = os.path.join(folder_path, actual_file)
                new_file_path = os.path.join(folder_path, correct_name)
                
                # 파일명을 변경
                try:
                    os.rename(old_file_path, new_file_path)
                    print(f"Renamed: {actual_file} -> {correct_name}")
                except Exception as e:
                    print(f"Error renaming {actual_file}: {e}")
                break
        else:
            # 목록에 있지만 실제 폴더에 없는 파일 처리
            print(f"No matching file found for: {correct_name}")


In [2]:
file_list_csv = 'list_onepiece.csv'  # 파일명 목록이 담긴 CSV 파일
folder_path = 'preprocessing_reviews'  # 실제 파일이 존재하는 폴더

rename_files_based_on_list(file_list_csv, folder_path)

No matching file found for: reviews_1875663.csv
No matching file found for: reviews_3977452.csv
No matching file found for: reviews_1875664.csv
No matching file found for: reviews_4018731.csv
No matching file found for: reviews_3977488.csv
No matching file found for: reviews_2551401.csv
No matching file found for: reviews_2978106.csv
No matching file found for: reviews_3295891.csv
No matching file found for: reviews_1645762.csv
No matching file found for: reviews_1944554.csv


In [45]:
file_list_csv = 'list_pants.csv'  # 파일명 목록이 담긴 CSV 파일
folder_path = 'reviews'  # 실제 파일이 존재하는 폴더

rename_files_based_on_list(file_list_csv, folder_path)

Renamed: Deep One Tuck Sweat Shorts [Grey]_reviews.csv -> Deep One Tuck Sweat Shorts [Grey]_reviews.csv
Renamed: 바이오워싱 카펜터 버뮤다 데님 팬츠_라이트블루_reviews.csv -> 바이오워싱 카펜터 버뮤다 데님 팬츠_라이트블루_reviews.csv
Renamed: 데미지 워시드 데님 팬츠-미디엄 블루(Cool Air)_reviews.csv -> 데미지 워시드 데님 팬츠-미디엄 블루(Cool Air)_reviews.csv
Renamed: Wide Cargo Half Denim Pants - 5COL_reviews.csv -> Wide Cargo Half Denim Pants - 5COL_reviews.csv
Renamed: Deep One Tuck Sweat Shorts [Black]_reviews.csv -> Deep One Tuck Sweat Shorts [Black]_reviews.csv
Renamed: [쿨탠다드] 세미 와이드 히든 밴딩 슬랙스 [블랙]_reviews.csv -> [쿨탠다드] 세미 와이드 히든 밴딩 슬랙스 [블랙]_reviews.csv
Renamed: 버뮤다 와이드 원턱 스웨트 쇼츠 [그레이]_reviews.csv -> 버뮤다 와이드 원턱 스웨트 쇼츠 [그레이]_reviews.csv
Renamed: 캣 브러쉬 워시드 버뮤다 데님 쇼츠 (SAND BLUE)_reviews.csv -> 캣 브러쉬 워시드 버뮤다 데님 쇼츠 (SAND BLUE)_reviews.csv
Renamed: 캣 브러쉬 워시드 버뮤다 데님 쇼츠 (NORMAL BLACK)_reviews.csv -> 캣 브러쉬 워시드 버뮤다 데님 쇼츠 (NORMAL BLACK)_reviews.csv
Renamed: Deep One Tuck Sweat Pants [Grey]_reviews.csv -> Deep One Tuck Sweat Pants [Grey]_reviews.csv


In [46]:
file_list_csv = 'list_jackets.csv'  # 파일명 목록이 담긴 CSV 파일
folder_path = 'reviews'  # 실제 파일이 존재하는 폴더

rename_files_based_on_list(file_list_csv, folder_path)

Renamed: [무료반품]베켄바우어 트랙탑 - 인디고_IP0418.csv -> [무료반품]베켄바우어 트랙탑 - 인디고_IP0418.csv
Renamed: 파이어버드 트랙탑 - 블랙_IJ7058.csv -> 파이어버드 트랙탑 - 블랙_IJ7058.csv
Renamed: ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이.csv -> ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이.csv
Renamed: [무료반품]베켄바우어 트랙탑 - 블랙화이트_II5763.csv -> [무료반품]베켄바우어 트랙탑 - 블랙화이트_II5763.csv
Renamed: 2WAY 스웻 후드 집업 (MELANGE GREY).csv -> 2WAY 스웻 후드 집업 (MELANGE GREY).csv
Renamed: [유튜버 PICK]FR-40S 프렌치 워크 5P 자켓_Military Navy.csv -> [유튜버 PICK]FR-40S 프렌치 워크 5P 자켓_Military Navy.csv
Renamed: [쿨탠다드] 릴렉스드 베이식 블레이저 [블랙].csv -> [쿨탠다드] 릴렉스드 베이식 블레이저 [블랙].csv
Renamed: 시어 후드 베스트 블랙.csv -> 시어 후드 베스트 블랙.csv
Renamed: [SAINT YEAR] 세미 오버 핏 셋업 수트_GREY.csv -> [SAINT YEAR] 세미 오버 핏 셋업 수트_GREY.csv
Renamed: MATIN KIM LOGO COATING JUMPER IN BLACK.csv -> MATIN KIM LOGO COATING JUMPER IN BLACK.csv


In [4]:
import os

# 파일이 위치한 디렉터리 경로 설정
directory = 'preprocessing_reviews'

# 디렉터리 내의 파일들을 하나씩 확인하며 반복
for filename in os.listdir(directory):
    # '_reviews'와 'reviews_'를 빈 문자열로 바꾸어 새 파일명 생성
    new_filename = filename.replace('_reviews', '').replace('reviews_', '')
    
    # 기존 파일 경로와 새로운 파일 경로 생성
    old_file = os.path.join(directory, filename)
    new_file = os.path.join(directory, new_filename)
    
    # 파일명 변경
    os.rename(old_file, new_file)

print("파일 이름 변경 완료.")

파일 이름 변경 완료.


In [11]:
df = pd.read_csv('list_tshirts.csv')
df

,File Name
0,DOODLE HEART HALF T WHITE GREYISH BLUE.csv
1,[SET] 워셔블 케이블 반팔 니트 세트.csv
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트.csv
3,[3천원 결제혜택]링클 체크 박시 오버핏 롤업 하프 셔츠 다크 네이비.csv
4,460G 컷 헤비 피그먼트 티셔츠-차콜-.csv
5,[16수 코마사] 2PACK SMALL ARCH T-SHIRT WHITE _ BLA...
6,빈티지 워싱 네이비 체크셔츠.csv
7,[3PACK] 에어쿨링 드라이 기능성 무지 반팔티.csv
8,TAG OG TEE - WHITE.csv
9,[2PACK] EL 스트리트 아트워크 오버핏 반팔티 2종 2PACK.csv


In [9]:
df['File Name'] = df['File Name'].str.replace('reviews_', '')
df.to_csv('list_onepiece.csv', index=False)